[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sandy-lee29/app-review/blob/main/review_scraper.ipynb)


## Week 1 : App review Data Extraction and Clean Up
Industry: Music

Companies Analyzed:
- Apple Music
- Amazon Music
- Spotify
- Youtube Music

In [ ]:
!pip install nltk emoji textblob
import pandas as pd
import csv
import re
import emoji
import nltk
from nltk.corpus import stopwords
from nltk import pos_tag
from textblob import TextBlob
from datetime import datetime, timedelta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 7.4 MB/s eta 0:00:00


## 1.1 Google App Store Data Extraction

In [ ]:
!pip install -q google-play-scraper
import pandas as pd
from google_play_scraper import reviews, Sort

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 540.3 kB/s eta 0:00:00


In [ ]:
def fetch_reviews(package_name: str, n_reviews: int = 1000) -> pd.DataFrame:
    """
    Fetch Google Play reviews for a given package using NEWEST and MOST_RELEVANT sorting.

    Args:
        package_name (str): The package name of the app on Google Play.
        n_reviews (int): The maximum number of reviews per sorting method (default: 1000).

    Returns:
        pd.DataFrame: A DataFrame containing raw review data.
    """
    # Fetch newest reviews (NEWEST)
    newest_reviews, _ = reviews(
        package_name,
        lang='en',
        country='us',
        sort=Sort.NEWEST,
        count=n_reviews,
        filter_score_with=None
    )

    # Fetch most relevant reviews (MOST_RELEVANT)
    relevant_reviews, _ = reviews(
        package_name,
        lang='en',
        country='us',
        sort=Sort.MOST_RELEVANT,
        count=n_reviews,
        filter_score_with=None
    )

    # Combine both datasets
    all_reviews_data = newest_reviews + relevant_reviews
    reviews_df = pd.DataFrame(all_reviews_data)

    return reviews_df

def clean_reviews(df: pd.DataFrame, app_name: str) -> pd.DataFrame:
    """
    Clean and preprocess the review data.

    Args:
        df (pd.DataFrame): Raw review DataFrame.
        app_name (str): Name of the app.

    Returns:
        pd.DataFrame: Cleaned DataFrame with only necessary columns.
                    Final columns: review_id, review, rating, time, company, data_source, app_name.
    """
    if df.empty:
        return df  # If no reviews, return empty DataFrame

    # Select only the necessary columns and make a copy
    df = df[["reviewId", "content", "score", "at"]].copy()

    # Rename columns
    df.rename(columns={
        "reviewId": "review_id",
        "content": "review",
        "score": "rating",
        "at": "time"
    }, inplace=True)

    # Add columns for the app name and data source
    df["company"] = app_name           # Add the app name as company
    df["data_source"] = "Android"      # Indicate the data source

    # Data Cleaning: Clean review text, convert rating to numeric, and time to datetime
    df["review"] = df["review"].astype(str)\
                    .str.replace(",", " ", regex=False)\
                    .str.replace("\n", " ", regex=False)\
                    .str.strip()
    df["rating"] = pd.to_numeric(df["rating"], errors='coerce')
    df["time"] = pd.to_datetime(df["time"], errors='coerce')

    # Remove rows with missing values in critical columns
    df.dropna(subset=["review", "rating", "time"], inplace=True)

    # Keep only valid ratings (1 to 5)
    df = df[(df["rating"] >= 1) & (df["rating"] <= 5)]

    # Remove duplicate reviews based on 'review_id'
    df.drop_duplicates(subset=["review_id"], inplace=True)

    return df

def scrape_multiple_apps(apps: dict, n_reviews: int = 1000, output_filename: str = "google_play_reviews.csv"):
    """
    Extract and save Google Play reviews for multiple apps.

    Args:
        apps (dict): Dictionary with app names as keys and package names as values.
        n_reviews (int): Number of reviews per sorting method.
        output_filename (str): CSV filename to save the reviews (default: "google_play_reviews.csv").
    """
    all_reviews = []

    for app_name, package_name in apps.items():
        print(f"Scraping reviews for {app_name}...")
        raw_reviews = fetch_reviews(package_name, n_reviews)
        cleaned_reviews = clean_reviews(raw_reviews, app_name)
        all_reviews.append(cleaned_reviews)

    # Combine all app reviews into a single DataFrame
    final_df = pd.concat(all_reviews, ignore_index=True)

    # Save to CSV
    final_df.to_csv(output_filename, index=False)
    print(f"All reviews saved successfully to {output_filename}! 🎉")

# Dictionary of top 4 Music Apps on Google Play
apps = {
    "Apple Music": "com.apple.android.music",
    "Spotify": "com.spotify.music",
    "Amazon Music": "com.amazon.mp3",
    "YouTube Music": "com.google.android.apps.youtube.music",
}

scrape_multiple_apps(apps)


Scraping reviews for Apple Music...
Scraping reviews for Spotify...
Scraping reviews for Amazon Music...
Scraping reviews for YouTube Music...
All reviews saved successfully to google_play_reviews.csv! 🎉


In [ ]:
google_reviews = pd.read_csv('google_play_reviews.csv')

In [ ]:
google_reviews.count()

,0
review_id,7474
review,7474
rating,7474
time,7474
company,7474
data_source,7474


## 1.2 Apple App Store Data Extraction

In [ ]:
pip install -q app-store-scraper
from app_store_scraper import AppStore

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully uninstalled urllib3-2.3.0
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled req

In [ ]:
def fetch_apple_reviews(app_name: str, app_id: str, n_reviews: int = 1000, **kwargs) -> pd.DataFrame:
    """
    Fetches Apple App Store reviews for the given app.

    Args:
        app_name (str): The name of the app.
        app_id (str): The unique identifier of the app.
        n_reviews (int): The number of reviews to fetch (default: 1000).
        **kwargs: Additional options:
            - sleep_milliseconds (int): Delay between requests in milliseconds (default: 2000ms).
            - lang (str): Review language (default: 'en').
            - country (str): Country code (default: 'us').

    Returns:
        pd.DataFrame: A DataFrame containing the fetched review data.
    """
    # Set default parameters
    default_params = {
        'sleep_milliseconds': 2000,
        'lang': 'en',
        'country': 'us',
        'count': n_reviews
    }
    params = {**default_params, **kwargs}

    # Initialize the AppStore scraper
    app = AppStore(
        country=params['country'],
        app_id=app_id,
        app_name=app_name
    )

    # Fetch reviews (limit the number using the how_many parameter)
    app.review(
        how_many=params['count'],
        sleep=params['sleep_milliseconds'] / 1000  # Convert milliseconds to seconds
    )

    # Convert the result to a DataFrame and return
    return pd.DataFrame(app.reviews)

def clean_apple_reviews(df: pd.DataFrame, app_name: str) -> pd.DataFrame:
    """
    Cleans and processes Apple App Store review data.

    Args:
        df (pd.DataFrame): The raw review DataFrame.
        app_name (str): The name of the app.

    Returns:
        pd.DataFrame: A cleaned DataFrame containing only the necessary columns,
                      with an empty 'review_id' column and the 'app_name' column added.
    """
    if df.empty:
        return df  # Return as is if no reviews are available

    # Select necessary columns (usually "review", "rating", and "date" are provided)
    expected_columns = ["review", "rating", "date"]
    available_cols = [col for col in expected_columns if col in df.columns]
    df = df[available_cols].copy()

    # Rename the 'date' column to 'time' if it exists
    if 'date' in df.columns:
        df.rename(columns={'date': 'time'}, inplace=True)

    # Clean the review text: remove commas, newlines, and trim whitespace
    if 'review' in df.columns:
        df['review'] = df['review'].astype(str)\
                            .str.replace(",", " ", regex=False)\
                            .str.replace("\n", " ", regex=False)\
                            .str.strip()

    # Convert ratings to numeric values
    if 'rating' in df.columns:
        df['rating'] = pd.to_numeric(df['rating'], errors='coerce')

    # Convert the time column to datetime
    if 'time' in df.columns:
        df['time'] = pd.to_datetime(df['time'], errors='coerce')

    # Drop rows with missing required values
    df.dropna(subset=['review', 'rating', 'time'], inplace=True)

    # Keep only valid ratings (1 to 5)
    df = df[(df['rating'] >= 1) & (df['rating'] <= 5)]

    # Remove duplicate reviews based on the review text
    df.drop_duplicates(subset=['review'], inplace=True)

    # Add columns: create an empty 'review_id', add app name and data source columns
    df["review_id"] = ""               # Create review_id column (empty)
    df["company"] = app_name
    df["data_source"] = "IOS"           # Indicate the data source

    # Optionally, reorder the columns
    final_columns = ["review_id", "review", "rating", "time", "company", "data_source"]
    df = df[final_columns]

    return df

def scrape_multiple_apple_apps(apps: dict, n_reviews: int = 1000, output_filename: str = "apple_app_reviews.csv"):
    """
    Fetches Apple App Store reviews for multiple apps and saves them to a CSV file.

    Args:
        apps (dict): A dictionary with app names as keys and app IDs as values.
        n_reviews (int): The number of reviews to fetch per app (default: 1000).
        output_filename (str): The CSV filename for saving the reviews.
    """
    all_reviews = []

    for app_name, app_id in apps.items():
        print(f"Scraping reviews for {app_name}...")
        # Fetch review data
        raw_reviews = fetch_apple_reviews(app_name, app_id, n_reviews=n_reviews)
        # Clean the review data
        cleaned_reviews = clean_apple_reviews(raw_reviews, app_name)
        all_reviews.append(cleaned_reviews)

    # Combine all app reviews into a single DataFrame
    final_df = pd.concat(all_reviews, ignore_index=True)

    # Save the DataFrame to a CSV file
    final_df.to_csv(output_filename, index=False)
    print(f"All reviews saved successfully to {output_filename}! 🎉")

# Dictionary of app names and their corresponding Apple App Store IDs
apps = {
    "Apple Music": "1108187390",
    "Spotify": "324684580",
    "Amazon Music": "571800810",
    "YouTube Music": "1017492454",
}

# Start scraping reviews
scrape_multiple_apple_apps(apps, n_reviews=1000)



Scraping reviews for Apple Music...
Scraping reviews for Spotify...
Scraping reviews for Amazon Music...
Scraping reviews for YouTube Music...
All reviews saved successfully to apple_app_reviews.csv! 🎉


In [ ]:
apple_reviews = pd.read_csv('apple_app_reviews.csv')

In [ ]:
apple_reviews.count()

,0
review_id,0
review,4000
rating,4000
time,4000
company,4000
data_source,4000


## 1.3 Combining Google + Apple Review and Preprocessing

In [ ]:
    # Download necessary NLTK resources (downloads will be skipped if already present)
    nltk.download('stopwords', quiet=True)
    nltk.download('punkt', quiet=True)
    nltk.download('averaged_perceptron_tagger', quiet=True)
    nltk.download('averaged_perceptron_tagger_eng', quiet=True)

True

In [ ]:
combined_reviews = pd.concat([google_reviews, apple_reviews], ignore_index=True)

In [ ]:
def preprocess_combined_reviews(df: pd.DataFrame, review_column: str = 'review') -> pd.DataFrame:
    """
    Preprocesses review texts in the given DataFrame by applying the following steps:
      1. Convert text to lowercase.
      2. Remove emojis.
      3. Remove special characters (e.g., punctuation and symbols).
      4. Remove meaningless reviews (e.g., reviews containing only adjectives or very short reviews).

    Args:
        df (pd.DataFrame): A DataFrame containing combined reviews from Google and Apple.
        review_column (str): The name of the column containing review texts (default is 'review').

    Returns:
        pd.DataFrame: A DataFrame with preprocessed review texts. Rows with empty reviews after processing are removed.
    """

    def remove_emojis(text: str) -> str:
        return emoji.replace_emoji(text, replace="")  # Remove emojis

    def remove_special_chars(text: str) -> str:
        return re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove all non-alphanumeric characters (except spaces)

    def remove_meaningless_reviews(text: str) -> str:
        words = text.split()
        pos_tags = pos_tag(words)  # Get part-of-speech tags
        # Extract adjectives (POS tags starting with 'JJ')
        adj_only = [word for word, tag in pos_tags if tag.startswith("JJ")]
        # Remove review if it only contains adjectives (with 5 or fewer words) or is very short (6 or fewer words)
        if (len(adj_only) == len(words) and len(words) <= 5) or len(words) <= 6:
            return ""
        return text

    def preprocess_review(text: str) -> str:
        if isinstance(text, str):
            text = text.lower()  # Convert to lowercase
            text = remove_emojis(text)
            text = remove_special_chars(text)
            text = remove_meaningless_reviews(text)
        return text

    # Apply the preprocessing to the specified review column
    df[review_column] = df[review_column].astype(str).apply(preprocess_review)

    # Remove any rows with empty reviews after processing
    df = df[df[review_column].str.strip() != ""]

    return df



In [ ]:
processed_reviews = preprocess_combined_reviews(combined_reviews, review_column="review")
processed_reviews.to_csv("cleaned_reviews.csv", index=False)
print("✅ Review preprocessing complete! Cleaned data saved as 'cleaned_reviews.csv'")


✅ Review preprocessing complete! Cleaned data saved as 'cleaned_reviews.csv'


In [ ]:
processed_reviews.count()

,0
review_id,5696
review,9695
rating,9695
time,9695
company,9695
data_source,9695
